In [92]:
from textblob import TextBlob
from bs4 import BeautifulSoup
from pyquery import PyQuery
from urllib.parse import urlparse
from urllib.error import HTTPError
from tika import parser
import json
import pprint
import requests
import sys
import os
import operator
import socket

In [93]:
### PARAMETERS ###
input_filename = 'au'
input_filetype = 'txt'
countryCode = 'AU'

In [95]:
def detect_language(unknownLanguageText):
    try:
        text = TextBlob(unknownLanguageText)    
        return text.detect_language()

    except:
        print('Unexpected error:', sys.exc_info()[0])
        return None

In [96]:
def scrape_policy_from_url(url):
    if url.endswith('.pdf'):
        return None
    try:
        headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:87.0) Gecko/20100101 Firefox/87.0'
        }
        req = requests.get(url, headers)

        soup = BeautifulSoup(req.content, 'html.parser')
        policy_text = soup.get_text().strip()     
        return policy_text
    except HTTPError as err:
        print('HTTPError:',err.code)
        return None
    except:
        print('Unexpected error:', sys.exc_info()[0])
    return None

In [97]:
def scrape_policy_from_pdf(url):
    try:
        raw = parser.from_file(url)    
        return raw['content']
    except:
        print('Unexpected error:', sys.exc_info()[0])
    return None    

In [98]:
def get_hosting_location(url):
    try:
        url = url.replace("https://","")
        url = url.replace("http://","")
        url = url.replace("www.", "") # May replace some false positives ('www.com')
        url = url.split('/')[0]
        ip = socket.gethostbyname(url)
        requesturl = 'http://ipwhois.app/json/'+ip
        r = requests.get(requesturl)
        if r.status_code == requests.codes.ok:
            pq = PyQuery(r.text)
            body = json.loads(pq.html())
        return body["country"]
    except:
        print('Unexpected error:', sys.exc_info()[0])
        
    return None

In [99]:
def enrich_jsons_with_policy_texts(json_data):    
    for element in json_data:
        if 'descriptionHTML' in element:
            del element['descriptionHTML']
        # mit textclean die description bereinigen        
        #if 'description' in element:
        #    element['description'] = element['description'].replace("\r\n","")
        if 'icon' in element:
            del element['icon']
        if 'headerImage' in element:
            del element['headerImage']
        if 'screenshots' in element:
            del element['screenshots']
        if 'comments' in element:
            del element['comments'] 
        if 'recentChanges' in element:
            del element['recentChanges']
        if 'privacyPolicy' in element:
            element['privacyPolicyText'] = scrape_policy_from_url(element['privacyPolicy'])
            element['privacyPolicyLanguage'] = detect_language(element['privacyPolicyText'])
            element['hostingLocation'] = get_hosting_location(element['privacyPolicy'])
            element['fromPlaystore'] = countryCode
    return json_data

In [100]:
def sort_json_data(json_data):    
    sorted_json = sorted(json_data, key=lambda k: k.get('maxInstalls', 0), reverse=True)      
    return sorted_json

In [101]:
def save_json_data_to_file(json_data):    
    with open(input_filename+"_enriched+sorted.json", "w") as testresult_file:
        json.dump(json_data, testresult_file, indent=4, sort_keys=True) 

In [102]:
with open(input_filename+'.'+input_filetype) as f:
    json_data = json.load(f)   


json_data = enrich_jsons_with_policy_texts(json_data)
json_data_sorted = sort_json_data(json_data)
save_json_data_to_file(json_data_sorted)
#pprint.pp(json_data_sorted)

Unexpected error: <class 'textblob.exceptions.TranslatorError'>
Unexpected error: <class 'requests.exceptions.TooManyRedirects'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'textblob.exceptions.TranslatorError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.SSLError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.SSLError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.ConnectionError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.ConnectionError'>
Unexpected error: <class 'TypeErr

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Unexpected error: <class 'urllib.error.HTTPError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.SSLError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.ConnectionError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'textblob.exceptions.TranslatorError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.SSLError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'requests.exceptions.ConnectionError'>
Unexpected error: <class 'TypeError'>
Unexpected error: <class 'socket.gaierror'>


In [113]:
## Concatenating all languages
#list_of_languages = ["ca", "de", "fr", "ie", "it", "jp","mt", "uk", "us"]

#import pandas as pd

#df = pd.read_json("au_enriched+sorted.json")
#print(df.shape[0])

#for lang in list_of_languages:
#    df_extension = pd.read_json(lang+"_enriched+sorted.json")
#    df = pd.concat([df, df_extension], ignore_index=True)

#print(df.shape[0])    

195
2191


In [114]:
#df.to_pickle("10countries.pkl")
#df.to_json("10countries.json")